In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!unzip gdrive/My\ Drive/Dataset/RAVDESS.zip

Mounted at /content/gdrive
Archive:  gdrive/My Drive/Dataset/RAVDESS.zip
   creating: Actor_01/
  inflating: Actor_01/03-01-01-01-01-01-01.wav  
  inflating: Actor_01/03-01-01-01-01-02-01.wav  
  inflating: Actor_01/03-01-01-01-02-01-01.wav  
  inflating: Actor_01/03-01-01-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-01-01-01-01.wav  
  inflating: Actor_01/03-01-02-01-01-02-01.wav  
  inflating: Actor_01/03-01-02-01-02-01-01.wav  
  inflating: Actor_01/03-01-02-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-02-01-01-01.wav  
  inflating: Actor_01/03-01-02-02-01-02-01.wav  
  inflating: Actor_01/03-01-02-02-02-01-01.wav  
  inflating: Actor_01/03-01-02-02-02-02-01.wav  
  inflating: Actor_01/03-01-03-01-01-01-01.wav  
  inflating: Actor_01/03-01-03-01-01-02-01.wav  
  inflating: Actor_01/03-01-03-01-02-01-01.wav  
  inflating: Actor_01/03-01-03-01-02-02-01.wav  
  inflating: Actor_01/03-01-03-02-01-01-01.wav  
  inflating: Actor_01/03-01-03-02-01-02-01.wav  
  inflating: Actor_01/

In [ ]:
!ls

Actor_01  Actor_05  Actor_09  Actor_13	Actor_17  Actor_21  gdrive
Actor_02  Actor_06  Actor_10  Actor_14	Actor_18  Actor_22  sample_data
Actor_03  Actor_07  Actor_11  Actor_15	Actor_19  Actor_23
Actor_04  Actor_08  Actor_12  Actor_16	Actor_20  Actor_24


In [ ]:
!pip install librosa soundfile

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# all the available emotions in RAVDESS Dataset
emotions = {
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

# emotions to be observed
observed_emotions = ['calm', 'happy', 'angry', 'disgust']

In [ ]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [ ]:
def load_data(test_size = 0.2):
  x, y = [], []
  for folder in glob.glob('/content/Actor_*'):
    print(folder)
    for file in glob.glob(folder + '/*.wav'):
      file_name = os.path.basename(file)
      emotion = emotions[file_name.split('-')[2]]
      if emotion not in observed_emotions:
        continue
      feature = extract_feature(file, mfcc = True, chroma = True, mel = True)
      x.append(feature)
      y.append(emotion)
  return train_test_split(np.array(x), y, test_size = test_size, random_state = 42)

In [ ]:
x_train, x_test, y_train, y_test = load_data(test_size=0.2)
# print some details
# number of samples in training data
print("Number of training samples:", x_train.shape[0])
# number of samples in testing data
print("Number of testing samples:", x_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using utils.extract_features() method
print("Number of features:", x_train.shape[1])

/content/Actor_18
/content/Actor_19
/content/Actor_03
/content/Actor_01
/content/Actor_08
/content/Actor_10
/content/Actor_16
/content/Actor_11
/content/Actor_09
/content/Actor_21
/content/Actor_17
/content/Actor_15
/content/Actor_20
/content/Actor_12
/content/Actor_22
/content/Actor_23
/content/Actor_24
/content/Actor_04
/content/Actor_07
/content/Actor_02
/content/Actor_06
/content/Actor_13
/content/Actor_05
/content/Actor_14
Number of training samples: 614
Number of testing samples: 154
Number of features: 180


## **MLP Classifier**

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(alpha = 0.001, batch_size = 250, epsilon = 1e-08, hidden_layer_sizes = (500,), learning_rate = 'adaptive', max_iter = 500)
model.fit(x_train, y_train)

MLPClassifier(activation='relu', alpha=0.001, batch_size=250, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(500,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 74.03%


In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.54      0.81      0.65        31
        calm       0.88      0.88      0.88        50
     disgust       0.71      0.58      0.64        26
       happy       0.81      0.64      0.71        47

    accuracy                           0.74       154
   macro avg       0.74      0.73      0.72       154
weighted avg       0.76      0.74      0.74       154



In [ ]:
matrix = confusion_matrix(y_test,y_pred)
print (matrix)

[[25  1  1  4]
 [ 1 44  3  2]
 [ 8  2 15  1]
 [12  3  2 30]]


## **Decision Tree Classifier**

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(max_depth = 6).fit(x_train, y_train) 
dtree_predictions = dtree_model.predict(x_test) 

In [ ]:
accuracy_dtree = accuracy_score(y_true=y_test,y_pred=dtree_predictions)
print("Accuracy: {:.2f}%".format(accuracy_dtree*100))

Accuracy: 53.90%


In [ ]:
print(classification_report(y_test,dtree_predictions)) 

              precision    recall  f1-score   support

       angry       0.49      0.74      0.59        31
        calm       0.86      0.64      0.74        50
     disgust       0.31      0.62      0.42        26
       happy       0.63      0.26      0.36        47

    accuracy                           0.54       154
   macro avg       0.57      0.56      0.53       154
weighted avg       0.63      0.54      0.54       154



In [ ]:
print(confusion_matrix(y_test, dtree_predictions))

[[23  0  6  2]
 [ 0 32 15  3]
 [ 5  3 16  2]
 [19  2 14 12]]


## **Support Vector Machine**

In [ ]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_train, y_train) 
svm_predictions = svm_model_linear.predict(x_test) 

In [ ]:
accuracy_svm = accuracy_score(y_true=y_test,y_pred=svm_predictions)
print ("Accuracy: {:.2f}%".format(accuracy_svm*100))

Accuracy: 66.23%


In [ ]:
print(classification_report(y_test,svm_predictions)) 

              precision    recall  f1-score   support

       angry       0.59      0.77      0.67        31
        calm       0.88      0.76      0.82        50
     disgust       0.44      0.73      0.55        26
       happy       0.78      0.45      0.57        47

    accuracy                           0.66       154
   macro avg       0.67      0.68      0.65       154
weighted avg       0.72      0.66      0.67       154



In [ ]:
print(confusion_matrix(y_test, svm_predictions))

[[24  0  4  3]
 [ 3 38  8  1]
 [ 3  2 19  2]
 [11  3 12 21]]


## **Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# create regressor object 
classifier = RandomForestClassifier(n_estimators = 100, random_state = 0)   
# fit the regressor with x and y data 
classifier.fit(x_train, y_train)   
c_p = classifier.predict(x_test) 

In [ ]:
accuracy_rfc = accuracy_score(y_true=y_test,y_pred=c_p)
print ("Accuracy: {:.2f}%".format(accuracy_rfc*100))

Accuracy: 71.43%


In [ ]:
print(classification_report(y_test,c_p)) 

              precision    recall  f1-score   support

       angry       0.69      0.71      0.70        31
        calm       0.93      0.86      0.90        50
     disgust       0.56      0.73      0.63        26
       happy       0.62      0.55      0.58        47

    accuracy                           0.71       154
   macro avg       0.70      0.71      0.70       154
weighted avg       0.73      0.71      0.72       154



In [ ]:
print(confusion_matrix(y_test,c_p))

[[22  0  4  5]
 [ 0 43  2  5]
 [ 0  1 19  6]
 [10  2  9 26]]
